In [1]:
base_directory = '../rawdata'
file_extension = "*.csv"

In [2]:
from scripts.utils import load_device_data

danmini_doorbell_df = load_device_data(base_directory, file_extension, 'Danmini_Doorbell')
ennio_doorbell_df = load_device_data(base_directory, file_extension, 'Ennio_Doorbell')
# Ecobee Thermostat
ecobee_thermostat_df = load_device_data(base_directory, file_extension, 'Ecobee_Thermostat')
# Web cam
samsung_cam_df = load_device_data(base_directory, file_extension, 'Samsung_SNH_1011_N_Webcam')
# Baby monitor
baby_monitor_df = load_device_data(base_directory, file_extension, 'Philips_B120N10_Baby_Monitor')
# Security cam
provision_cam1_df = load_device_data(base_directory, file_extension, 'Provision_PT_737E_Security_Camera')
provision_cam2_df = load_device_data(base_directory, file_extension, 'Provision_PT_838_Security_Camera')
simplehome_cam1_df = load_device_data(base_directory, file_extension, 'SimpleHome_XCS7_1002_WHT_Security_Camera')
simplehome_cam2_df = load_device_data(base_directory, file_extension, 'SimpleHome_XCS7_1003_WHT_Security_Camera')

In [3]:
dataframe = {"Danmin Door Bell": danmini_doorbell_df, 
            "Ecobee_Thermostat": ecobee_thermostat_df,
            "Ennio_Doorbell": ennio_doorbell_df,
            "Philips_B120N10_Baby_Monitor": baby_monitor_df,
            "Provision_PT_737E_Security_Camera": provision_cam1_df,
            "Provision_PT_838_Security_Camera": provision_cam2_df,
            "Samsung_SNH_1011_N_Webcam": samsung_cam_df,
            "SimpleHome_XCS7_1002_WHT_Security_Camera": simplehome_cam1_df,
            "SimpleHome_XCS7_1003_WHT_Security_Camera": simplehome_cam2_df
            }

In [4]:
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn.metrics import precision_score, recall_score, f1_score # <-- Added this import

# Helper function to create the sliding windows
def create_sequences(X, y, time_steps=10):
    """
    Creates sequences from time-series data.
    
    X: Input features (scaled)
    y: Input labels (encoded)
    time_steps: How many previous rows to use for prediction
    """
    Xs, ys = [], []
    
    for i in range(len(X) - time_steps):
        # Get a window of 'time_steps' rows
        v = X[i:(i + time_steps)]
        # Get the label for the *end* of that window
        Xs.append(v)
        ys.append(y[i + time_steps])
        
    return np.array(Xs), np.array(ys)

# --- Your Main Function (Revised) ---

def cnn_lstm_classifier(data, device_name, n_steps=10):
    """
    Trains a CNN-LSTM classifier with correct data preprocessing
    and calculates precision, recall, and f1-score.
    
    data: Your input pandas DataFrame
    device_name: String name for saving the model
    n_steps: The number of time steps (rows) to look back (hyperparameter)
    """
    
    print("Starting model training...")
    
    # --- 1. Initial Data Prep ---
    # Separate features and labels
    X = data.drop(['label', 'device'], axis=1)
    y = data['label']
    
    # Get feature count *before* scaling
    n_features = X.shape[1]
    print(f"Original data shape: {X.shape}")

    # Encode labels (e.g., 'benign' -> 0, 'attack_1' -> 1)
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    # Get the number of unique classes
    n_classes = len(np.unique(y_encoded))
    print(f"Found {n_features} features and {n_classes} classes.")

    # --- 2. Stratified Split FIRST ---
    # We split *before* creating sequences to prevent data leakage
    # Stratify ensures class proportions are maintained in train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, 
        test_size=0.3, 
        random_state=42,
        stratify=y_encoded  # <-- CRITICAL for imbalanced data
    )

    # --- 3. Scale Data ---
    # LSTMs need scaled data. Fit *only* on training data.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # --- 4. Create Sequences ---
    # Now we create the sliding windows (e.g., [samples, 10, 115])
    print(f"Creating sequences with n_steps={n_steps}...")
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, n_steps)
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test, n_steps)

    if X_train_seq.shape[0] == 0:
        print("Error: No sequences created. Is n_steps larger than your data?")
        return "Training failed."
        
    print(f"New training data shape: {X_train_seq.shape}") # Should be [samples, n_steps, n_features]
    print(f"New training label shape: {y_train_seq.shape}") # Should be [samples]

    # --- 5. One-Hot Encode Labels ---
    # We do this *after* creating sequences and splitting
    y_train_cat = to_categorical(y_train_seq, num_classes=n_classes)
    y_test_cat = to_categorical(y_test_seq, num_classes=n_classes)

    # --- 6. Handle Class Imbalance (Optional but Recommended) ---
    # This tells the model to pay more attention to rare classes
    # 'balanced' mode automatically calculates weights
    cw = class_weight.compute_class_weight(
        'balanced',
        classes=np.unique(y_train_seq), # Use the sequence labels
        y=y_train_seq
    )
    class_weights = dict(enumerate(cw))
    print(f"Calculated class weights: {class_weights}")

    # --- 7. Build and Compile Model ---
    model = Sequential()
    # The input shape is (timesteps, features)
    model.add(Conv1D(
        filters=64, 
        kernel_size=3, 
        activation='relu', 
        input_shape=(n_steps, n_features)  # <-- THE KEY CHANGE
    ))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    # Output layer has one neuron per class
    model.add(Dense(n_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    # --- 8. Train the Model ---
    history = model.fit(
        X_train_seq, 
        y_train_cat, 
        epochs=10,  # You may need more epochs
        batch_size=64, 
        validation_data=(X_test_seq, y_test_cat),
        class_weight=class_weights,  # <-- Use the class weights
        verbose=1
    )

    # --- 9. Evaluate and Save ---
    print("\nEvaluating model on test data...")
    loss, accuracy = model.evaluate(X_test_seq, y_test_cat, verbose=0)
    
    # --- 10. Calculate Precision, Recall, F1-Score (NEW) ---
    print("Calculating additional metrics...")
    y_pred_prob = model.predict(X_test_seq)
    y_pred = np.argmax(y_pred_prob, axis=1)
    
    # We compare against the original integer-encoded test labels (y_test_seq)
    # Use 'weighted' average for imbalanced classes, matches class_weight logic
    precision = precision_score(y_test_seq, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test_seq, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test_seq, y_pred, average='weighted', zero_division=0)
    
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Precision (Weighted): {precision:.4f}")
    print(f"Test Recall (Weighted): {recall:.4f}")
    print(f"Test F1-Score (Weighted): {f1:.4f}")

    results = {
        'Accuracy': accuracy,
        'Loss': loss,
        'Precision_Weighted': precision, # <-- New
        'Recall_Weighted': recall,       # <-- New
        'F1_Score_Weighted': f1,         # <-- New
        'n_steps': n_steps,
        'features': n_features
    }

    modelpath = f'models/{device_name}'
    if not os.path.exists(modelpath):
        os.makedirs(modelpath) # Use makedirs for nested paths
    
    model.save(f'{modelpath}/{device_name}_cnn_lstm_model.h5')

    return f'CNN-LSTM Model trained and saved successfully \n {results}'


In [5]:
for k in dataframe:
    print("----------------------xxxxxxx----------------------")
    print(k)
    print("----------------------xxxxxxx----------------------")
    
    
    # New CNN-LSTM Classifier
    results_cnn_lstm = cnn_lstm_classifier(dataframe[k], k)
    print("CNN-LSTM Results:")
    print(results_cnn_lstm)
    
    print("---------------------xxxxxxx-----------------------")

----------------------xxxxxxx----------------------
Danmin Door Bell
----------------------xxxxxxx----------------------
Starting model training...
Original data shape: (1018298, 115)
Found 115 features and 3 classes.
Creating sequences with n_steps=10...
New training data shape: (712798, 10, 115)
New training label shape: (712798,)
Calculated class weights: {0: np.float64(6.850599236898), 1: np.float64(1.0719476175867275), 2: np.float64(0.5205226564548131)}


c:\Users\hp\Desktop\My Work\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 8, 64)          │        22,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,447 (345.50 KB)

...

c:\Users\hp\Desktop\My Work\venv\Lib\site-packages\keras\src\ops\nn.py:938: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\hp\Desktop\My Work\venv\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


214/214 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accur

c:\Users\hp\Desktop\My Work\venv\Lib\site-packages\keras\src\ops\nn.py:938: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


Test Accuracy: 1.0000
Test Loss: 0.0000
Test Precision (Weighted): 1.0000
Test Recall (Weighted): 1.0000
Test F1-Score (Weighted): 1.0000
CNN-LSTM Results:
CNN-LSTM Model trained and saved successfully 
 {'Accuracy': 1.0, 'Loss': 0.0, 'Precision_Weighted': 1.0, 'Recall_Weighted': 1.0, 'F1_Score_Weighted': 1.0, 'n_steps': 10, 'features': 115}
---------------------xxxxxxx-----------------------
